# COMP9033 - Data Analytics Lab 08a: Decision tree classification
## Introduction

This lab focuses on SMS message spam detection using decision tree and random forest classification. It's a direct counterpart to the rule-based spam detection from Lab 05 and the $k$ nearest neighbours spam detector from Lab 07a. At the end of the lab, you should be able to use `scikit-learn` to:

- Create a decision tree classification model and a random forest classification model.
- Use the models to predict new values.
- Measure the accuracy of the models.

### Getting started

Let's start by importing the packages we'll need. As usual, we'll import `pandas` for exploratory analysis, but this week we're also going to use the `tree` subpackage from `scikit-learn` to create decision tree models and the `ensemble` subpackage to create random forest models.

In [ ]:
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

Next, let's load the data. Write the path to your sms.csv file in the cell below:

In [ ]:
data_file = 'data/sms.csv'

Execute the cell below to load the CSV data into a pandas data frame with the columns `label` and `message`.

> **Note:** This week, the CSV file is not comma separated, but instead tab separated. We can tell `pandas` about the different format using the `sep` argument, as shown in the cell below. For more information, see the `read_csv` [documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html).

In [ ]:
sms = pd.read_csv(data_file, sep='\t', header=None, names=['label', 'message'])
sms.head()

## Data modelling

Let's start by splitting our data into training and test sets:

In [ ]:
X = sms['message']
y = sms['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=0)

### Decision tree classification

Next, let's build a decision tree classification model of the SMS message data. `scikit-learn` supports decision tree functionality via the [`tree`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree) subpackage. This subpackage supports both decision tree regression and classification. We can use the [`DecisionTreeClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) class to build our model.

`DecisionTreeClassifier` accepts a number of different hyperparameters and the model we build may be more or less accurate depending on their values. We can get a list of these modelling parameters using the `get_params` method of the estimator (this works on any `scikit-learn` estimator), like this:

In [ ]:
DecisionTreeClassifier().get_params()

You can find a more detailed description of each parameter in the `scikit-learn` [documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier). 

Let's use model selection via cross validation to select the optimal decision tree classification model from a set of candidates:

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS),
    DecisionTreeClassifier(random_state=0) # Control randomness, so the lab works the same for everyone
)

parameters = {
    'decisiontreeclassifier__criterion': ['gini', 'entropy'],
    'decisiontreeclassifier__min_samples_leaf': [1, 2, 3, 4, 5, 10, 15, 20],
    'decisiontreeclassifier__min_samples_split': [2, 3, 4, 5, 6, 10, 15, 20]
}

gs = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1) # n_jobs=-1 uses all available CPUs for calculation
gs.fit(X_train, y_train) # Fit using the training set

y_pred = gs.predict(X_test) # Make predictions using the test set

print(classification_report(y_test, y_pred)) # Print the classification report

The model is much more accurate than both the dummy model above and the rule-based model from lab 05. Specifically, we can say that:

- 95% of the messages we labelled as ham were actually ham (precision for ham = 0.95).
- 87% of the messages we labelled as spam were actually spam (precision for spam = 0.87).
- We labelled 98% of ham as ham (recall for ham = 0.98).
- We labelled 69% of spam as spam (recall for spam = 0.69).

This is a significant increase in spam detection ability over the nearest neighbours model from Lab 07a, where recall for spam was just 42%. However, it comes at the cost of decreased precision for spam (100% to 87%), i.e. 13% of the messages we classify as spam are actually ham. Such a high rate is likely to be unacceptable in practice.

### Random forest classification

Let's build a random forest classification model of the SMS message data to see if we can improve over the simple decision tree model. `scikit-learn` supports ensemble model functionality via the [`ensemble`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.ensemble) subpackage. This subpackage supports random forest regression and classification, as well as several other ensemble modelling algorithms. We can use the [`RandomForestClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) class to build our model.

When building random forest classifiers, we can choose to specify the same hyperparameters as with decision tree classifiers. But let's keep it simple and just specify the number of subtrees to include in the forest (feel free to adjust the grid search parameters, if you want to). We can do this using the `n_estimators` attribute. Let's use model selection to choose the best random forest classifier from a range of different values of `n_estimators`:

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS),
    RandomForestClassifier(random_state=0) # Control randomness, so the lab works the same for everyone
)

parameters = {
    'randomforestclassifier__n_estimators': [5, 10, 15, 20, 25, 35, 45, 50]
}

gs = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1) # n_jobs=-1 uses all available CPUs for calculation
gs.fit(X_train, y_train) # Fit using the training set

y_pred = gs.predict(X_test) # Make predictions using the test set

print(classification_report(y_test, y_pred)) # Print the classification report

As can be seen, the random forest model is the most accurate spam detector that we've built so far. Specifically, we can say that:

- 96% of the messages we labelled as ham were actually ham (precision for ham = 0.96).
- 99% of the messages we labelled as spam were actually spam (precision for spam = 0.99).
- We labelled every actual ham as ham (recall for ham = 1.00).
- We labelled 76% of spam as spam (recall for spam = 0.76).

Our spam detection rate (spam recall) has now increased to 76%, while our misclassification rate is just 1% (although this still means that we misclassify one in one hundred messages). By tweaking the hyperparameters or adding new preprocessing stages, we could increase the accuracy of our model even more.